# Postprocess offshore wind met-ocean conditions (JONSWAP)

In [1]:
%%capture 

# Important libraries header information
# ----------------------------------

# change this directory as necessary
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['/ascldap/users/lcheung/wind_uq/amrwind-frontend/',
                 '/ccs/proj/cfd162/lcheung/amrwind-frontend/']
import sys, os, shutil
for x in amrwindfedirs: sys.path.insert(1, x)

# Load the libraries
import matplotlib.pyplot as plt
import amrwind_frontend  as amrwind
import numpy             as np
from matplotlib import cm
import re
import time

import pandas as pd
from io import StringIO

# Also ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Make all plots inline 
%matplotlib inline

In [2]:
def plotrotorlines(zhh, rotorD, xstart, xend, **kwargs):
    plt.hlines(zhh,             xstart, xend, linewidth=1.0, **kwargs) # Plot the HH line
    plt.hlines(zhh+rotorD*0.5,  xstart, xend, linewidth=0.5, **kwargs) # Plot the HH line
    plt.hlines(zhh-rotorD*0.5,  xstart, xend, linewidth=0.5, **kwargs) # Plot the HH line  

## Postprocess and compare AMR-Wind velocity profiles

In [3]:
 # Set your run directory here
#casedir = '/nscratch/lcheung/2021/SWIFT_Neutral_WS8.7_Alpha0.14/AMRWind_Precursor/precursor1'
rundir='/lustre/orion/cfd162/scratch/lcheung/ALCC_Frontier_WindFarm/MedWS_LowTI'
caselist = [   
            {'rundir':rundir+'/precursor6_4kmX2km_5m', 'tag':'precursor6 4x2',
             'inputfile':'MedWS_LowTI_precursor1.inp',
             'ncfile':'post_processing/abl_statistics45000.nc', 'avgtimes':[25000, 30000],
             'mstyle':{'mfc':'y', 'marker':'^', 'lw':0, 'mec':'k',}, 'lstyle':{'color':'orange', 'ls':':'}},           
            {'rundir':rundir+'/precursor6_7kmX2km_5m', 'tag':'precursor6 7x2',
             'inputfile':'MedWS_LowTI_Offshore_Stable_Precursor_Coarse.inp',
             'ncfile':'post_processing/abl_statistics00000.nc', 'avgtimes':[25000, 30000],
             'mstyle':{'mfc':'y', 'marker':'v', 'lw':0, 'mec':'k',}, 'lstyle':{'color':'orange', 'ls':'-.'}},           
]

# Average between 15,000 sec to 20,000 sec
avgtimes = [25000, 30000]

# Hub-height locations
plotheights=[30, 90, 150, 270]

zHH_target    = 150.0
rotorD        = 240
bottom_tip    = zHH_target - 0.5*rotorD
top_tip       = zHH_target + 0.5*rotorD

# Load the entire netcdf in memory
loadinmemory = False   # Do this only if there's enough RAM and for new (python 3+ netCDF4) libraries

In [4]:
for case in caselist:
    case['App'] = amrwind.MyApp.init_nogui()
    tstart = time.time()
    case['App'].ABLpostpro_loadnetcdffile(case['rundir']+'/'+case['ncfile'], usemmap=loadinmemory)
    tend   = time.time()
    print("Load time: %f sec"%(tend-tstart))

Loading /lustre/orion/cfd162/scratch/lcheung/ALCC_Frontier_WindFarm/MedWS_LowTI/precursor6_4kmX2km_5m/post_processing/abl_statistics45000.nc
Time range: 18000.800000 to 30000.000000
Done.
Load time: 6.224638 sec
Loading /lustre/orion/cfd162/scratch/lcheung/ALCC_Frontier_WindFarm/MedWS_LowTI/precursor6_7kmX2km_5m/post_processing/abl_statistics00000.nc
Time range: 1.000000 to 30000.000000
Done.
Load time: 12.464236 sec


In [5]:
# First, let's look at the averaged statistics
for case in caselist:
    print("***** "+case['tag']+" *******")
    tstart = time.time()
    case['reportstats'] = case['App'].ABLpostpro_printreport(avgt=case['avgtimes'], avgz=plotheights,span=(bottom_tip,top_tip))
    tend   = time.time()
    print("Compute time: %f sec"%(tend-tstart))

***** precursor6 4x2 *******
Loading w'theta'_r
Loading w'w'_r
Loading v
Loading v'v'_r
Loading theta
Loading u
Loading u'u'_r
        z       Uhoriz      WindDir       TI_TKE     TI_horiz        Alpha    Alpha-Fit     ObukhovL         Veer     Veer-Fit 
      ===         ====         ====         ====         ====         ====         ====         ====         ====         ==== 
    30.00 6.929304e+00 2.641424e+02 4.610156e-02 7.485359e-02 1.388864e-01 1.751899e-01 2.584811e+02 5.673989e-02 5.268938e-02 
    90.00 8.214372e+00 2.668964e+02 3.130968e-02 4.894094e-02 1.723174e-01 1.751899e-01 3.361394e+02 4.835241e-02 5.268938e-02 
   150.00 9.029968e+00 2.700017e+02 2.394898e-02 3.746787e-02 1.862016e-01 1.751899e-01 5.327082e+02 5.477850e-02 5.268938e-02 
   270.00 9.991643e+00 2.766277e+02 1.484978e-02 2.330630e-02 1.326986e-01 1.751899e-01 1.570657e+03 5.174361e-02 5.268938e-02 

ustar: 0.212511
Compute time: 15.223085 sec
***** precursor6 7x2 *******
Loading w'theta'_r
Loading w'w'

## Load met ocean conditions
Download the data from: https://www.nrel.gov/wind/nwtc/metocean-data.html and the excel sheet at 
https://www.nrel.gov/wind/nwtc/assets/downloads/MetOcean/DistributionParameters.xlsx
Read the paper: https://onlinelibrary.wiley.com/doi/abs/10.1002/we.1881

In [6]:
JONSWAP_param = """
WindSpeed,SigWaveHeight,PeakSpectralPeriod,Directionality
4.00,	1.10,			8.52,			-20.81915116
6.00,	1.18,			8.31,			-28.5540023
8.00,	1.32,			8.01,			-27.38277538
10.00,	1.54,			7.65,			-19.40350485
12.00,	1.84,			7.44,			-12.28992154
14.00,	2.19,			7.46,			-10.39386558
16.00,	2.60,			7.64,			-7.153822365
18.00,	3.06,			8.05,			-4.813508643
20.00,	3.62,			8.52,			-1.548944606
22.00,	4.03,			8.99,			5.434583753
24.00,	4.52,			9.45,			7.044655005
"""

In [7]:
df = pd.read_csv(StringIO(JONSWAP_param))
#print(np.interp(8.22, df['WindSpeed'], df['SigWaveHeight']))
print(df)

    WindSpeed  SigWaveHeight  PeakSpectralPeriod  Directionality
0         4.0           1.10                8.52      -20.819151
1         6.0           1.18                8.31      -28.554002
2         8.0           1.32                8.01      -27.382775
3        10.0           1.54                7.65      -19.403505
4        12.0           1.84                7.44      -12.289922
5        14.0           2.19                7.46      -10.393866
6        16.0           2.60                7.64       -7.153822
7        18.0           3.06                8.05       -4.813509
8        20.0           3.62                8.52       -1.548945
9        22.0           4.03                8.99        5.434584
10       24.0           4.52                9.45        7.044655


In [8]:
for case in caselist:
    print("***** "+case['tag']+" *******")
    WS90=float(case['reportstats']['Uhoriz'][1])
    print('SigWaveHeight = %f'%np.interp(WS90, df['WindSpeed'], df['SigWaveHeight']))
    print('PeakSpectralPeriod = %f'%np.interp(WS90, df['WindSpeed'], df['PeakSpectralPeriod']))
    print('Directionality = %f'%np.interp(WS90, df['WindSpeed'], df['Directionality']))
    #print(WS90)

***** precursor6 4x2 *******
SigWaveHeight = 1.343581
PeakSpectralPeriod = 7.971413
Directionality = -26.527507
***** precursor6 7x2 *******
SigWaveHeight = 1.344040
PeakSpectralPeriod = 7.970661
Directionality = -26.510842
